### IMPORTS

In [1]:
# load, preprocess, scale, baseline
from wrangle import final_dataset

# manipulate data
import numpy as np

# visualize data
import matplotlib.pyplot as plt

# neural network weights
from modelsNN import good_weights

# optmization problem
from modelsRO import good_problem

np.random.seed(123)

### Load, Clean, Preprocess, Scale, Baseline: Cardiovascular Disease

In [2]:
X_train_scaled_cd, X_test_scaled_cd, y_train_cd, y_test_cd = final_dataset(dataset='cvd')

CVD Loading and Cleaning...
CVD Loaded and Cleaned...

CVD Splitting...
CVD Split...

CVD Scaling...
CVD Scaled...



### Neural Network Comparisons: Cardiovascular Disease

In [3]:
%%time
#nn_algo_tune = good_weights(X_train_scaled_cd, y_train_cd, X_test_scaled_cd, y_test_cd)

# Copy all dataframes to csvs
# import pandas as pd
# for a in nn_algo_tune:
#     for c in nn_algo_tune[a]:
#         df = pd.DataFrame()
#         for b in nn_algo_tune[a][c]:
#             print(a, c, b, nn_algo_tune[a][c][b])
#             df[b] = [nn_algo_tune[a][c][b]]
#         print(dd)
#         path = f"/home/leonardo_leads/Documents/SchoolDocs/ga_tech_masters/omscs_ml/spring2024/cs7641_machine_learning/lectures/week9/assignment2/source/plots/nn_algo_tune/{a}/{c}/{c}.csv"
#         df.to_csv(path_or_buf=path, index=False)
#         print('********************')

Running Random Hill Climbing
Get_NN_Algo: Random Hill Climbing
Instantiated Random Hill Climbing
Time (Seconds): 3.910064697265625e-05

Working on: Random Hill Climbing
Fitting: Random Hill Climbing
Model Fitting Complete. Time: 32.2075355052948 seconds

Predicting (TRAINING): Random Hill Climbing

Calculating Recall (TRAINING): Random Hill Climbing

Appending Results (TRAINING): Random Hill Climbing

Recall Score (TRAINING): 0.5462026804608512%

Predicting (TESTING): Random Hill Climbing

Calculting Recall (TESTING): Random Hill Climbing

Appending Results (TESTING): Random Hill Climbing

Recall Score (TESTING): 0.5712383488681758%

Running Random Hill Climbing
Get_NN_Algo: Random Hill Climbing
Instantiated Random Hill Climbing
Time (Seconds): 1.811981201171875e-05

Working on: Random Hill Climbing
Fitting: Random Hill Climbing
Model Fitting Complete. Time: 64.73573207855225 seconds

Predicting (TRAINING): Random Hill Climbing

Calculating Recall (TRAINING): Random Hill Climbing

Appe

### Analysis: Neural Network Comparisons

In [771]:
lr = 'learning_rate'
ps = 'pop_size'
mp = 'mutation_prob'
sch = 'schedule'
rs = 'restarts'
for algo in nn_algo_tune:
    for combo in nn_algo_tune[algo]:
        nnac = nn_algo_tune[algo][combo]
        i = combo[-1]
        match algo:
            case "Random Hill Climbing":
                algo_combo_train = f"rhc{i}_train = 'recall train', {nnac['recall_train']}"
                algo_combo_test = f"rhc{i}_test = 'recall test', {nnac['recall_test']}"
                algo_combo_lr = f"rhc{i}_lr = '{lr}', {nnac[lr]}"
                algo_combo_rs = f"rhc{i}_rs = '{rs}', {nnac[rs]}"
                exec(algo_combo_rs)
            case "Simulated Annealing":
                algo_combo_train = f"sa{i}_train = 'recall train', {nnac['recall_train']}"
                algo_combo_test = f"sa{i}_test = 'recall test', {nnac['recall_test']}"
                algo_combo_lr = f"sa{i}_lr = '{lr}', {nnac[lr]}"
                algo_combo_sch = f"sa{i}_sch = '{sch}', {nnac[sch]}"
                exec(algo_combo_sch)
            case "Genetic Algorithm":
                algo_combo_train = f"ga{i}_train = 'recall train', {nnac['recall_train']}"
                algo_combo_test = f"ga{i}_test = 'recall test', {nnac['recall_test']}"
                algo_combo_lr = f"ga{i}_lr = '{lr}', {nnac[lr]}"
                algo_combo_ps = f"ga{i}_ps = '{ps}', {nnac[ps]}"
                algo_combo_mp = f"ga{i}_mp = '{mp}', {nnac[mp]}"
                exec(algo_combo_ps)
                exec(algo_combo_mp)
            case "Gradient Descent":
                algo_combo_train = f"gd_train = 'recall train', {nnac['recall_train']}"
                algo_combo_test = f"gd_test = 'recall test', {nnac['recall_test']}"
        exec(algo_combo_train)
        exec(algo_combo_test)
        exec(algo_combo_lr)

# random hill climbing
rhc_data = [(rhc1_lr[1], rhc1_rs[1], rhc1_train[1], rhc1_test[1]),
        (rhc2_lr[1], rhc2_rs[1], rhc2_train[1], rhc2_test[1]),
        (rhc3_lr[1], rhc3_rs[1], rhc3_train[1], rhc1_test[1])]
rhc_cols = [rhc1_lr[0], rhc1_rs[0], rhc1_train[0], rhc1_test[0]]
rhc_df = pd.DataFrame(data=rhc_data, columns=rhc_cols)

# simulated annealing
sa_data = [(sa1_lr[1], sa1_sch[1], 'Geom', sa1_train[1], sa1_test[1]),
        (sa2_lr[1], sa2_sch[1], 'Arith', sa2_train[1], sa2_test[1]),
        (sa3_lr[1], sa3_sch[1], 'Exp', sa3_train[1], sa3_test[1])]
sa_cols = [sa1_lr[0], sa1_sch[0], 'Decay', sa1_train[0], sa1_test[0]]
sa_df = pd.DataFrame(data=sa_data, columns=sa_cols)

# genetic algorithm
ga_data = [(ga1_lr[1], ga1_ps[1], ga1_mp[1], ga1_train[1], ga1_test[1]),
        (ga2_lr[1], ga2_ps[1], ga2_mp[1], ga2_train[1], ga2_test[1]),
        (ga3_lr[1], ga3_ps[1], ga3_mp[1], ga3_train[1], ga3_test[1])]
ga_cols = [ga1_lr[0], ga1_ps[0], ga1_mp[0], ga1_train[0], ga1_test[0]]
ga_df = pd.DataFrame(data=ga_data, columns=ga_cols)

# gradient descent
gd_data = [(gd_train[1], gd_test[1])]
gd_cols = [gd_train[0], gd_test[0]]
gd_df = pd.DataFrame(data=gd_data, columns=gd_cols)

### Plot Neural Network Comparisons

In [820]:
rhc_df, sa_df, ga_df, gd_df

(   learning_rate  restarts  recall train  recall test
 0           0.01         5      0.546203     0.571238
 1           0.05        10      0.567129     0.592543
 2           0.10        15      0.498237     0.571238,
    learning_rate  schedule  Decay  recall train  recall test
 0           0.01      50.0   Geom      0.542441     0.564581
 1           0.05      50.0  Arith      0.574653     0.593875
 2           0.10      50.0    Exp      0.596755     0.629827,
    learning_rate  pop_size  mutation_prob  recall train  recall test
 0           0.01       175           0.05      0.674348     0.668442
 1           0.05       200           0.10      0.680226     0.677763
 2           0.10       225           0.15      0.652716     0.644474,
    recall train  recall test
 0      0.539854     0.556591)

### Optimization Problem: RHC, SA, GA, MIMIC

In [4]:
%%time
#opt_prob = good_problem()

# Copy all dataframes to csvs
# for a in opt_prob:
#     for c in opt_prob[a]:
#         for df in opt_prob[a][c]['results']:
#             print(a, c, df.keys())
#             for k in df.keys():
#                 print(k)
#                 path = f"/home/leonardo_leads/Documents/SchoolDocs/ga_tech_masters/omscs_ml/spring2024/cs7641_machine_learning/lectures/week9/assignment2/source/plots/opt_prob/{a}/{c}/{k}.csv"
#                 df[k].to_csv(path_or_buf=path, index=False)
#             print('********************')

Running All Algorithms

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 0.3187445640563965 minutes.
Running Random Hill Climbing FlipFlop
Completed Random Hill Climbing FlipFlop. Time: 0.32537227074305214 minutes.
Running Random Hill Climbing Knapsack
Completed Random Hill Climbing Knapsack. Time: 0.31682502031326293 minutes.
Completed Random Hill Climbing. Time: 0.9609418551127116 minutes

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 0.3213028152783712 minutes.
Running Random Hill Climbing FlipFlop
Completed Random Hill Climbing FlipFlop. Time: 0.32963773409525554 minutes.
Running Random Hill Climbing Knapsack
Completed Random Hill Climbing Knapsack. Time: 0.32008928855260216 minutes.
Completed Random Hill Climbing. Time: 0.971029837926229 minutes

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing Fo

### Analysis: Optimization Problem

$$
\textbf{Random Hill Climbing: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [642]:
# seed 1

# boolean masks
bool1 = opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool2 = opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool3 = opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FP']['Restarts'] == 15

rhc_combo1FP = (opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FP'][bool1]
    .reset_index(drop=True))
rhc_combo2FP = (opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FP'][bool2]
    .reset_index(drop=True))
rhc_combo3FP = (opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FP'][bool3]
    .reset_index(drop=True))

# seed 2

# boolean masks
bool4 = opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool5 = opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool6 = opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FP']['Restarts'] == 15

rhc_combo4FP = (opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FP'][bool4]
    .reset_index(drop=True))
rhc_combo5FP = (opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FP'][bool5]
    .reset_index(drop=True))
rhc_combo6FP = (opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FP'][bool6]
    .reset_index(drop=True))

# seed 3

# boolean masks
bool7 = opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool8 = opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool9 = opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FP']['Restarts'] == 15

rhc_combo7FP = (opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FP'][bool7]
    .reset_index(drop=True))
rhc_combo8FP = (opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FP'][bool8]
    .reset_index(drop=True))
rhc_combo9FP = (opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FP'][bool9]
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(rhc_combo1FP.index.max() + 1,
                      rhc_combo4FP.index.max() + 1,
                      rhc_combo7FP.index.max() + 1)

max_index_ps2 = max(rhc_combo2FP.index.max() + 1,
                      rhc_combo5FP.index.max() + 1,
                      rhc_combo8FP.index.max() + 1)

max_index_ps3 = max(rhc_combo3FP.index.max() + 1,
                      rhc_combo6FP.index.max() + 1,
                      rhc_combo9FP.index.max() + 1)

rhc_combo1FP = rhc_combo1FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
rhc_combo4FP = rhc_combo4FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
rhc_combo7FP = rhc_combo7FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

rhc_combo2FP = rhc_combo2FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
rhc_combo5FP = rhc_combo5FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
rhc_combo8FP = rhc_combo8FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

rhc_combo3FP = rhc_combo3FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
rhc_combo6FP = rhc_combo6FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
rhc_combo9FP = rhc_combo9FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

rhc_ps1avg = (rhc_combo1FP[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo4FP[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo7FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
rhc_ps2avg = (rhc_combo2FP[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo5FP[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo8FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
rhc_ps3avg = (rhc_combo3FP[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo6FP[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo9FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
rhc_ps1_FP = pd.concat([rhc_combo1FP, rhc_combo4FP, rhc_combo7FP, rhc_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
rhc_ps2_FP = pd.concat([rhc_combo2FP, rhc_combo5FP, rhc_combo8FP, rhc_ps2avg], axis=1)

# # Common Problem Size 3; Different Seeds
rhc_ps3_FP = pd.concat([rhc_combo3FP, rhc_combo6FP, rhc_combo9FP, rhc_ps3avg], axis=1)

$$
\textbf{Random Hill Climbing: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [643]:
# seed 1

# boolean masks
bool1 = opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool2 = opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool3 = opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FF']['Restarts'] == 15

rhc_combo1FF = (opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FF'][bool1]
    .reset_index(drop=True))
rhc_combo2FF = (opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FF'][bool2]
    .reset_index(drop=True))
rhc_combo3FF = (opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FF'][bool3]
    .reset_index(drop=True))

# seed 2

# boolean masks
bool4 = opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool5 = opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool6 = opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FF']['Restarts'] == 15

rhc_combo4FF = (opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FF'][bool4]
    .reset_index(drop=True))
rhc_combo5FF = (opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FF'][bool5]
    .reset_index(drop=True))
rhc_combo6FF = (opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FF'][bool6]
    .reset_index(drop=True))

# seed 3

# boolean masks
bool7 = opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool8 = opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool9 = opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FF']['Restarts'] == 15

rhc_combo7FF = (opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FF'][bool7]
    .reset_index(drop=True))
rhc_combo8FF = (opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FF'][bool8]
    .reset_index(drop=True))
rhc_combo9FF = (opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FF'][bool9]
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(rhc_combo1FF.index.max() + 1,
                      rhc_combo4FF.index.max() + 1,
                      rhc_combo7FF.index.max() + 1)

max_index_ps2 = max(rhc_combo2FF.index.max() + 1,
                      rhc_combo5FF.index.max() + 1,
                      rhc_combo8FF.index.max() + 1)

max_index_ps3 = max(rhc_combo3FF.index.max() + 1,
                      rhc_combo6FF.index.max() + 1,
                      rhc_combo9FF.index.max() + 1)

rhc_combo1FF = rhc_combo1FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
rhc_combo4FF = rhc_combo4FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
rhc_combo7FF = rhc_combo7FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

rhc_combo2FF = rhc_combo2FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
rhc_combo5FF = rhc_combo5FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
rhc_combo8FF = rhc_combo8FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

rhc_combo3FF = rhc_combo3FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
rhc_combo6FF = rhc_combo6FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
rhc_combo9FF = rhc_combo9FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

rhc_ps1avg = (rhc_combo1FF[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo4FF[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo7FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
rhc_ps2avg = (rhc_combo2FF[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo5FF[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo8FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
rhc_ps3avg = (rhc_combo3FF[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo6FF[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo9FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
rhc_ps1_FF = pd.concat([rhc_combo1FF, rhc_combo4FF, rhc_combo7FF, rhc_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
rhc_ps2_FF = pd.concat([rhc_combo2FF, rhc_combo5FF, rhc_combo8FF, rhc_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
rhc_ps3_FF = pd.concat([rhc_combo3FF, rhc_combo6FF, rhc_combo9FF, rhc_ps3avg], axis=1)

$$
\textbf{Random Hill Climbing: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [644]:
# seed 1

# boolean masks
bool1 = opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool2 = opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool3 = opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_KS']['Restarts'] == 15

rhc_combo1KS = (opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_KS'][bool1]
    .reset_index(drop=True))
rhc_combo2KS = (opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_KS'][bool2]
    .reset_index(drop=True))
rhc_combo3KS = (opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_KS'][bool3]
    .reset_index(drop=True))

# seed 2

# boolean masks
bool4 = opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool5 = opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool6 = opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_KS']['Restarts'] == 15

rhc_combo4KS = (opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_KS'][bool4]
    .reset_index(drop=True))
rhc_combo5KS = (opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_KS'][bool5]
    .reset_index(drop=True))
rhc_combo6KS = (opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_KS'][bool6]
    .reset_index(drop=True))

# seed 3

# boolean masks
bool7 = opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool8 = opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool9 = opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_KS']['Restarts'] == 15

rhc_combo7KS = (opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_KS'][bool7]
    .reset_index(drop=True))
rhc_combo8KS = (opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_KS'][bool8]
    .reset_index(drop=True))
rhc_combo9KS = (opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_KS'][bool9]
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(rhc_combo1KS.index.max() + 1,
                      rhc_combo4KS.index.max() + 1,
                      rhc_combo7KS.index.max() + 1)

max_index_ps2 = max(rhc_combo2KS.index.max() + 1,
                      rhc_combo5KS.index.max() + 1,
                      rhc_combo8KS.index.max() + 1)

max_index_ps3 = max(rhc_combo3KS.index.max() + 1,
                      rhc_combo6KS.index.max() + 1,
                      rhc_combo9KS.index.max() + 1)

rhc_combo1KS = rhc_combo1KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
rhc_combo4KS = rhc_combo4KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
rhc_combo7KS = rhc_combo7KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

rhc_combo2KS = rhc_combo2KS.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
rhc_combo5KS = rhc_combo5KS.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
rhc_combo8KS = rhc_combo8KS.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

rhc_combo3KS = rhc_combo3KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
rhc_combo6KS = rhc_combo6KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
rhc_combo9KS = rhc_combo9KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

rhc_ps1avg = (rhc_combo1KS[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo4KS[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo7KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
rhc_ps2avg = (rhc_combo2KS[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo5KS[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo8KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
rhc_ps3avg = (rhc_combo3KS[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo6KS[['Iteration', 'Time', 'Fitness','FEvals']] + rhc_combo9KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
rhc_ps1_KS = pd.concat([rhc_combo1KS, rhc_combo4KS, rhc_combo7KS, rhc_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
rhc_ps2_KS = pd.concat([rhc_combo2KS, rhc_combo5KS, rhc_combo8KS, rhc_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
rhc_ps3_KS = pd.concat([rhc_combo3KS, rhc_combo6KS, rhc_combo9KS, rhc_ps3avg], axis=1)

# Fix Column Names
rhc_list = [# FOUR PEAKS
rhc_ps1_FP,
rhc_ps2_FP,
rhc_ps3_FP,

# FLIP FLOP
rhc_ps1_FF,
rhc_ps2_FF,
rhc_ps3_FF,

# KNAPSACK
rhc_ps1_KS,
rhc_ps2_KS,
rhc_ps3_KS]

for rhc in rhc_list:
    rhc.columns = ['s1_Iteration', 's1_Time', 's1_Fitness', 's1_FEvals', 's1_Restarts', 's1_max_iters',
       's1_current_restart', 's2_Iteration', 's2_Time', 's2_Fitness', 's2_FEvals', 's2_Restarts',
       's2_max_iters', 's2_current_restart', 's3_Iteration', 's3_Time', 's3_Fitness',
       's3_FEvals', 's3_Restarts', 's3_max_iters', 's3_current_restart', 'avg_Iteration', 'avg_Time', 'avg_Fitness',
       'avg_FEvals']

$$
\textbf{Simulated Annleaing: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [645]:

# seed 1
sa_combo1FP = (opt_prob['Simulated Annealing']['sa_combo1']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo2FP = (opt_prob['Simulated Annealing']['sa_combo2']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo3FP = (opt_prob['Simulated Annealing']['sa_combo3']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 2
sa_combo4FP = (opt_prob['Simulated Annealing']['sa_combo4']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo5FP = (opt_prob['Simulated Annealing']['sa_combo5']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo6FP = (opt_prob['Simulated Annealing']['sa_combo6']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 3
sa_combo7FP = (opt_prob['Simulated Annealing']['sa_combo7']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo8FP = (opt_prob['Simulated Annealing']['sa_combo8']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo9FP = (opt_prob['Simulated Annealing']['sa_combo9']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(sa_combo1FP.index.max() + 1,
                      sa_combo4FP.index.max() + 1,
                      sa_combo7FP.index.max() + 1)

max_index_ps2 = max(sa_combo2FP.index.max() + 1,
                      sa_combo5FP.index.max() + 1,
                      sa_combo8FP.index.max() + 1)

max_index_ps3 = max(sa_combo3FP.index.max() + 1,
                      sa_combo6FP.index.max() + 1,
                      sa_combo9FP.index.max() + 1)

sa_combo1FP = sa_combo1FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
sa_combo4FP = sa_combo4FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
sa_combo7FP = sa_combo7FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

sa_combo2FP = sa_combo2FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
sa_combo5FP = sa_combo5FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
sa_combo8FP = sa_combo8FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

sa_combo3FP = sa_combo3FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
sa_combo6FP = sa_combo6FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
sa_combo9FP = sa_combo9FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

sa_ps1avg = (sa_combo1FP[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo4FP[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo7FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
sa_ps2avg = (sa_combo2FP[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo5FP[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo8FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
sa_ps3avg = (sa_combo3FP[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo6FP[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo9FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
sa_ps1_FP = pd.concat([sa_combo1FP, sa_combo4FP, sa_combo7FP, sa_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
sa_ps2_FP = pd.concat([sa_combo2FP, sa_combo5FP, sa_combo8FP, sa_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
sa_ps3_FP = pd.concat([sa_combo3FP, sa_combo6FP, sa_combo9FP, sa_ps3avg], axis=1)

$$
\textbf{Simulated Annleaing: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [646]:

# seed 1
sa_combo1FF = (opt_prob['Simulated Annealing']['sa_combo1']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo2FF = (opt_prob['Simulated Annealing']['sa_combo2']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo3FF = (opt_prob['Simulated Annealing']['sa_combo3']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 2
sa_combo4FF = (opt_prob['Simulated Annealing']['sa_combo4']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo5FF = (opt_prob['Simulated Annealing']['sa_combo5']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo6FF = (opt_prob['Simulated Annealing']['sa_combo6']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 3
sa_combo7FF = (opt_prob['Simulated Annealing']['sa_combo7']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo8FF = (opt_prob['Simulated Annealing']['sa_combo8']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo9FF = (opt_prob['Simulated Annealing']['sa_combo9']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(sa_combo1FF.index.max() + 1,
                      sa_combo4FF.index.max() + 1,
                      sa_combo7FF.index.max() + 1)

max_index_ps2 = max(sa_combo2FF.index.max() + 1,
                      sa_combo5FF.index.max() + 1,
                      sa_combo8FF.index.max() + 1)

max_index_ps3 = max(sa_combo3FF.index.max() + 1,
                      sa_combo6FF.index.max() + 1,
                      sa_combo9FF.index.max() + 1)

sa_combo1FF = sa_combo1FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
sa_combo4FF = sa_combo4FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
sa_combo7FF = sa_combo7FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

sa_combo2FF = sa_combo2FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
sa_combo5FF = sa_combo5FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
sa_combo8FF = sa_combo8FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

sa_combo3FF = sa_combo3FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
sa_combo6FF = sa_combo6FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
sa_combo9FF = sa_combo9FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

sa_ps1avg = (sa_combo1FF[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo4FF[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo7FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
sa_ps2avg = (sa_combo2FF[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo5FF[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo8FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
sa_ps3avg = (sa_combo3FF[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo6FF[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo9FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
sa_ps1_FF = pd.concat([sa_combo1FF, sa_combo4FF, sa_combo7FF, sa_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
sa_ps2_FF = pd.concat([sa_combo2FF, sa_combo5FF, sa_combo8FF, sa_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
sa_ps3_FF = pd.concat([sa_combo3FF, sa_combo6FF, sa_combo9FF, sa_ps3avg], axis=1)

$$
\textbf{Simulated Annleaning: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [647]:

# seed 1
sa_combo1KS = (opt_prob['Simulated Annealing']['sa_combo1']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo2KS = (opt_prob['Simulated Annealing']['sa_combo2']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo3KS = (opt_prob['Simulated Annealing']['sa_combo3']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 2
sa_combo4KS = (opt_prob['Simulated Annealing']['sa_combo4']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo5KS = (opt_prob['Simulated Annealing']['sa_combo5']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo6KS = (opt_prob['Simulated Annealing']['sa_combo6']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 3
sa_combo7KS = (opt_prob['Simulated Annealing']['sa_combo7']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo8KS = (opt_prob['Simulated Annealing']['sa_combo8']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo9KS = (opt_prob['Simulated Annealing']['sa_combo9']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(sa_combo1KS.index.max() + 1,
                      sa_combo4KS.index.max() + 1,
                      sa_combo7KS.index.max() + 1)

max_index_ps2 = max(sa_combo2KS.index.max() + 1,
                      sa_combo5KS.index.max() + 1,
                      sa_combo8KS.index.max() + 1)

max_index_ps3 = max(sa_combo3KS.index.max() + 1,
                      sa_combo6KS.index.max() + 1,
                      sa_combo9KS.index.max() + 1)

sa_combo1KS = sa_combo1KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
sa_combo4KS = sa_combo4KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
sa_combo7KS = sa_combo7KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

sa_combo2KS = sa_combo2KS.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
sa_combo5KS = sa_combo5KS.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
sa_combo8KS = sa_combo8KS.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

sa_combo3KS = sa_combo3KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
sa_combo6KS = sa_combo6KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
sa_combo9KS = sa_combo9KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

sa_ps1avg = (sa_combo1KS[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo4KS[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo7KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
sa_ps2avg = (sa_combo2KS[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo5KS[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo8KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
sa_ps3avg = (sa_combo3KS[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo6KS[['Iteration', 'Time', 'Fitness','FEvals']] + sa_combo9KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
sa_ps1_KS = pd.concat([sa_combo1KS, sa_combo4KS, sa_combo7KS, sa_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
sa_ps2_KS = pd.concat([sa_combo2KS, sa_combo5KS, sa_combo8KS, sa_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
sa_ps3_KS = pd.concat([sa_combo3KS, sa_combo6KS, sa_combo9KS, sa_ps3avg], axis=1)

sa_list = [# FOUR PEAKS
sa_ps1_FP,
sa_ps2_FP,
sa_ps3_FP,

# FLIP FLOP
sa_ps1_FF,
sa_ps2_FF,
sa_ps3_FF,

# KNAPsaCK
sa_ps1_KS,
sa_ps2_KS,
sa_ps3_KS]

for sa in sa_list:
    sa.columns = ['s1_Iteration', 's1_Time', 's1_Fitness', 's1_FEvals', 's1_Temperature', 's1_max_iters'
                  , 's2_Iteration', 's2_Time', 's2_Fitness', 's2_FEvals', 's2_Temperature',
       's2_max_iters', 's3_Iteration', 's3_Time', 's3_Fitness',
       's3_FEvals', 's3_Temperature', 's3_max_iters', 'avg_Iteration', 'avg_Time', 'avg_Fitness',
       'avg_FEvals']

    sa['s1_Temperature'] = sa['s1_Temperature'].astype('string').astype('int64')
    sa['s2_Temperature'] = sa['s2_Temperature'].astype('string').astype('int64')
    sa['s3_Temperature'] = sa['s3_Temperature'].astype('string').astype('int64')

$$
\textbf{Genetic Algorithm: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [648]:

# seed 1
ga_combo1FP = (opt_prob['Genetic Algorithm']['ga_combo1']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo2FP = (opt_prob['Genetic Algorithm']['ga_combo2']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo3FP = (opt_prob['Genetic Algorithm']['ga_combo3']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 2
ga_combo4FP = (opt_prob['Genetic Algorithm']['ga_combo4']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo5FP = (opt_prob['Genetic Algorithm']['ga_combo5']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo6FP = (opt_prob['Genetic Algorithm']['ga_combo6']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 3
ga_combo7FP = (opt_prob['Genetic Algorithm']['ga_combo7']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo8FP = (opt_prob['Genetic Algorithm']['ga_combo8']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo9FP = (opt_prob['Genetic Algorithm']['ga_combo9']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(ga_combo1FP.index.max() + 1,
                      ga_combo4FP.index.max() + 1,
                      ga_combo7FP.index.max() + 1)

max_index_ps2 = max(ga_combo2FP.index.max() + 1,
                      ga_combo5FP.index.max() + 1,
                      ga_combo8FP.index.max() + 1)

max_index_ps3 = max(ga_combo3FP.index.max() + 1,
                      ga_combo6FP.index.max() + 1,
                      ga_combo9FP.index.max() + 1)

ga_combo1FP = ga_combo1FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
ga_combo4FP = ga_combo4FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
ga_combo7FP = ga_combo7FP.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

ga_combo2FP = ga_combo2FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
ga_combo5FP = ga_combo5FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
ga_combo8FP = ga_combo8FP.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

ga_combo3FP = ga_combo3FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
ga_combo6FP = ga_combo6FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
ga_combo9FP = ga_combo9FP.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

ga_ps1avg = (ga_combo1FP[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo4FP[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo7FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
ga_ps2avg = (ga_combo2FP[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo5FP[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo8FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
ga_ps3avg = (ga_combo3FP[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo6FP[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo9FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
ga_ps1_FP = pd.concat([ga_combo1FP, ga_combo4FP, ga_combo7FP, ga_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
ga_ps2_FP = pd.concat([ga_combo2FP, ga_combo5FP, ga_combo8FP, ga_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
ga_ps3_FP = pd.concat([ga_combo3FP, ga_combo6FP, ga_combo9FP, ga_ps3avg], axis=1)

$$
\textbf{Genetic Algorithm: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [649]:

# seed 1
ga_combo1FF = (opt_prob['Genetic Algorithm']['ga_combo1']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo2FF = (opt_prob['Genetic Algorithm']['ga_combo2']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo3FF = (opt_prob['Genetic Algorithm']['ga_combo3']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 2
ga_combo4FF = (opt_prob['Genetic Algorithm']['ga_combo4']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo5FF = (opt_prob['Genetic Algorithm']['ga_combo5']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo6FF = (opt_prob['Genetic Algorithm']['ga_combo6']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 3
ga_combo7FF = (opt_prob['Genetic Algorithm']['ga_combo7']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo8FF = (opt_prob['Genetic Algorithm']['ga_combo8']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo9FF = (opt_prob['Genetic Algorithm']['ga_combo9']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(ga_combo1FF.index.max() + 1,
                      ga_combo4FF.index.max() + 1,
                      ga_combo7FF.index.max() + 1)

max_index_ps2 = max(ga_combo2FF.index.max() + 1,
                      ga_combo5FF.index.max() + 1,
                      ga_combo8FF.index.max() + 1)

max_index_ps3 = max(ga_combo3FF.index.max() + 1,
                      ga_combo6FF.index.max() + 1,
                      ga_combo9FF.index.max() + 1)

ga_combo1FF = ga_combo1FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
ga_combo4FF = ga_combo4FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
ga_combo7FF = ga_combo7FF.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

ga_combo2FF = ga_combo2FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
ga_combo5FF = ga_combo5FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)
ga_combo8FF = ga_combo8FF.reindex(index=range(0, max_index_ps2),
                                    fill_value=0)

ga_combo3FF = ga_combo3FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
ga_combo6FF = ga_combo6FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
ga_combo9FF = ga_combo9FF.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

ga_ps1avg = (ga_combo1FF[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo4FF[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo7FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
ga_ps2avg = (ga_combo2FF[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo5FF[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo8FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
ga_ps3avg = (ga_combo3FF[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo6FF[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo9FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
ga_ps1_FF = pd.concat([ga_combo1FF, ga_combo4FF, ga_combo7FF, ga_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
ga_ps2_FF = pd.concat([ga_combo2FF, ga_combo5FF, ga_combo8FF, ga_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
ga_ps3_FF = pd.concat([ga_combo3FF, ga_combo6FF, ga_combo9FF, ga_ps3avg], axis=1)

$$
\textbf{Genetic Algorithm: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [650]:

# seed 1
ga_combo1KS = (opt_prob['Genetic Algorithm']['ga_combo1']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo2KS = (opt_prob['Genetic Algorithm']['ga_combo2']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo3KS = (opt_prob['Genetic Algorithm']['ga_combo3']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 2
ga_combo4KS = (opt_prob['Genetic Algorithm']['ga_combo4']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo5KS = (opt_prob['Genetic Algorithm']['ga_combo5']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo6KS = (opt_prob['Genetic Algorithm']['ga_combo6']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 3
ga_combo7KS = (opt_prob['Genetic Algorithm']['ga_combo7']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo8KS = (opt_prob['Genetic Algorithm']['ga_combo8']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo9KS = (opt_prob['Genetic Algorithm']['ga_combo9']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(ga_combo1KS.index.max() + 1,
                      ga_combo4KS.index.max() + 1,
                      ga_combo7KS.index.max() + 1)

max_index_p2 = max(ga_combo2KS.index.max() + 1,
                      ga_combo5KS.index.max() + 1,
                      ga_combo8KS.index.max() + 1)

max_index_ps3 = max(ga_combo3KS.index.max() + 1,
                      ga_combo6KS.index.max() + 1,
                      ga_combo9KS.index.max() + 1)

ga_combo1KS = ga_combo1KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
ga_combo4KS = ga_combo4KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)
ga_combo7KS = ga_combo7KS.reindex(index=range(0, max_index_ps1),
                                    fill_value=0)

ga_combo2KS = ga_combo2KS.reindex(index=range(0, max_index_p2),
                                    fill_value=0)
ga_combo5KS = ga_combo5KS.reindex(index=range(0, max_index_p2),
                                    fill_value=0)
ga_combo8KS = ga_combo8KS.reindex(index=range(0, max_index_p2),
                                    fill_value=0)

ga_combo3KS = ga_combo3KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
ga_combo6KS = ga_combo6KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)
ga_combo9KS = ga_combo9KS.reindex(index=range(0, max_index_ps3),
                                    fill_value=0)

ga_ps1avg = (ga_combo1KS[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo4KS[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo7KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
ga_p2avg = (ga_combo2KS[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo5KS[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo8KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
ga_ps3avg = (ga_combo3KS[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo6KS[['Iteration', 'Time', 'Fitness','FEvals']] + ga_combo9KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
ga_ps1_KS = pd.concat([ga_combo1KS, ga_combo4KS, ga_combo7KS, ga_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
ga_ps2_KS = pd.concat([ga_combo2KS, ga_combo5KS, ga_combo8KS, ga_p2avg], axis=1)

# Common Problem Size 3; Different Seeds
ga_ps3_KS = pd.concat([ga_combo3KS, ga_combo6KS, ga_combo9KS, ga_ps3avg], axis=1)

ga_list = [# FOUR PEAKS
ga_ps1_FP,
ga_ps2_FP,
ga_ps3_FP,

# FLIP FLOP
ga_ps1_FF,
ga_ps2_FF,
ga_ps3_FF,

# KNAPgaCK
ga_ps1_KS,
ga_ps2_KS,
ga_ps3_KS]

for ga in ga_list:
    ga.columns = ['s1_Iteration', 's1_Time', 's1_Fitness', 's1_FEvals', 's1_Population Size', 's1_Mutation Rate', 's1_max_iters',
            's2_Iteration', 's2_Time', 's2_Fitness', 's2_FEvals', 's2_Population Size', 's2_Mutation Rate', 's2_max_iters',
            's3_Iteration', 's3_Time', 's3_Fitness', 's3_FEvals', 's3_Population Size', 's3_Mutation Rate', 's3_max_iters', 'avg_Iteration',
            'avg_Time', 'avg_Fitness', 'avg_FEvals']

$$
\textbf{Mutual-Information-Maximizing Input Clustering: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [651]:

# seed 1
mimic_combo1FP = (opt_prob['MIMIC']['mimic_combo1']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo2FP = (opt_prob['MIMIC']['mimic_combo2']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo3FP = (opt_prob['MIMIC']['mimic_combo3']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 2
mimic_combo4FP = (opt_prob['MIMIC']['mimic_combo4']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo5FP = (opt_prob['MIMIC']['mimic_combo5']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo6FP = (opt_prob['MIMIC']['mimic_combo6']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 3
mimic_combo7FP = (opt_prob['MIMIC']['mimic_combo7']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo8FP = (opt_prob['MIMIC']['mimic_combo8']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo9FP = (opt_prob['MIMIC']['mimic_combo9']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(mimic_combo1FP.index.max() + 1,
                      mimic_combo4FP.index.max() + 1,
                      mimic_combo7FP.index.max() + 1)

max_index_ps2 = max(mimic_combo2FP.index.max() + 1,
                      mimic_combo5FP.index.max() + 1,
                      mimic_combo8FP.index.max() + 1)

max_index_ps3 = max(mimic_combo3FP.index.max() + 1,
                      mimic_combo6FP.index.max() + 1,
                      mimic_combo9FP.index.max() + 1)

mimic_combo1FP = mimic_combo1FP.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)   
mimic_combo4FP = mimic_combo4FP.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)   
mimic_combo7FP = mimic_combo7FP.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)   

mimic_combo2FP = mimic_combo2FP.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)   
mimic_combo5FP = mimic_combo5FP.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)   
mimic_combo8FP = mimic_combo8FP.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)   

mimic_combo3FP = mimic_combo3FP.reindex(index=range(0, max_index_ps3),
                                        fill_value=0)
mimic_combo6FP = mimic_combo6FP.reindex(index=range(0, max_index_ps3),
                                        fill_value=0)
mimic_combo9FP = mimic_combo9FP.reindex(index=range(0, max_index_ps3),
                                        fill_value=0)

mimic_ps1avg = (mimic_combo1FP[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo4FP[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo7FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
mimic_ps2avg = (mimic_combo2FP[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo5FP[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo8FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
mimic_ps3avg = (mimic_combo3FP[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo6FP[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo9FP[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
mimic_ps1_FP = pd.concat([mimic_combo1FP, mimic_combo4FP, mimic_combo7FP, mimic_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
mimic_ps2_FP = pd.concat([mimic_combo2FP, mimic_combo5FP, mimic_combo8FP, mimic_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
mimic_ps3_FP = pd.concat([mimic_combo3FP, mimic_combo6FP, mimic_combo9FP, mimic_ps3avg], axis=1)

$$
\textbf{Mutual-Information-Maximizing Input Clustering: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [652]:

# seed 1
mimic_combo1FF = (opt_prob['MIMIC']['mimic_combo1']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo2FF = (opt_prob['MIMIC']['mimic_combo2']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo3FF = (opt_prob['MIMIC']['mimic_combo3']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 2
mimic_combo4FF = (opt_prob['MIMIC']['mimic_combo4']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo5FF = (opt_prob['MIMIC']['mimic_combo5']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo6FF = (opt_prob['MIMIC']['mimic_combo6']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 3
mimic_combo7FF = (opt_prob['MIMIC']['mimic_combo7']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo8FF = (opt_prob['MIMIC']['mimic_combo8']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo9FF = (opt_prob['MIMIC']['mimic_combo9']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(mimic_combo1FF.index.max() + 1,
                      mimic_combo4FF.index.max() + 1,
                      mimic_combo7FF.index.max() + 1)

max_index_ps2 = max(mimic_combo2FF.index.max() + 1,
                      mimic_combo5FF.index.max() + 1,
                      mimic_combo8FF.index.max() + 1)

max_index_p3 = max(mimic_combo3FF.index.max() + 1,
                      mimic_combo6FF.index.max() + 1,
                      mimic_combo9FF.index.max() + 1)

mimic_combo1FF = mimic_combo1FF.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)
mimic_combo4FF = mimic_combo4FF.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)
mimic_combo7FF = mimic_combo7FF.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)

mimic_combo2FF = mimic_combo2FF.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)
mimic_combo5FF = mimic_combo5FF.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)
mimic_combo8FF = mimic_combo8FF.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)

mimic_combo3FF = mimic_combo3FF.reindex(index=range(0, max_index_p3),
                                        fill_value=0)
mimic_combo6FF = mimic_combo6FF.reindex(index=range(0, max_index_p3),
                                        fill_value=0)
mimic_combo9FF = mimic_combo9FF.reindex(index=range(0, max_index_p3),
                                        fill_value=0)

mimic_ps1avg = (mimic_combo1FF[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo4FF[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo7FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
mimic_ps2avg = (mimic_combo2FF[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo5FF[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo8FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
mimic_p3avg = (mimic_combo3FF[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo6FF[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo9FF[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
mimic_ps1_FF = pd.concat([mimic_combo1FF, mimic_combo4FF, mimic_combo7FF, mimic_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
mimic_ps2_FF = pd.concat([mimic_combo2FF, mimic_combo5FF, mimic_combo8FF, mimic_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
mimic_ps3_FF = pd.concat([mimic_combo3FF, mimic_combo6FF, mimic_combo9FF, mimic_p3avg], axis=1)

$$
\textbf{Mutual-Information-Maximizing Input Clustering: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [653]:

# seed 1
mimic_combo1KS = (opt_prob['MIMIC']['mimic_combo1']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo2KS = (opt_prob['MIMIC']['mimic_combo2']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo3KS = (opt_prob['MIMIC']['mimic_combo3']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 2
mimic_combo4KS = (opt_prob['MIMIC']['mimic_combo4']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo5KS = (opt_prob['MIMIC']['mimic_combo5']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo6KS = (opt_prob['MIMIC']['mimic_combo6']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 3
mimic_combo7KS = (opt_prob['MIMIC']['mimic_combo7']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo8KS = (opt_prob['MIMIC']['mimic_combo8']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo9KS = (opt_prob['MIMIC']['mimic_combo9']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# Average Seed
max_index_ps1 = max(mimic_combo1KS.index.max() + 1,
                      mimic_combo4KS.index.max() + 1,
                      mimic_combo7KS.index.max() + 1)

max_index_ps2 = max(mimic_combo2KS.index.max() + 1,
                      mimic_combo5KS.index.max() + 1,
                      mimic_combo8KS.index.max() + 1)

max_index_ps3 = max(mimic_combo3KS.index.max() + 1,
                      mimic_combo6KS.index.max() + 1,
                      mimic_combo9KS.index.max() + 1)

mimic_combo1KS = mimic_combo1KS.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)
mimic_combo4KS = mimic_combo4KS.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)
mimic_combo7KS = mimic_combo7KS.reindex(index=range(0, max_index_ps1),
                                        fill_value=0)

mimic_combo2KS = mimic_combo2KS.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)
mimic_combo5KS = mimic_combo5KS.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)
mimic_combo8KS = mimic_combo8KS.reindex(index=range(0, max_index_ps2),
                                        fill_value=0)

mimic_combo3KS = mimic_combo3KS.reindex(index=range(0, max_index_ps3),
                                        fill_value=0)
mimic_combo6KS = mimic_combo6KS.reindex(index=range(0, max_index_ps3),
                                        fill_value=0)
mimic_combo9KS = mimic_combo9KS.reindex(index=range(0, max_index_ps3),
                                        fill_value=0)

mimic_ps1avg = (mimic_combo1KS[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo4KS[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo7KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
mimic_ps2avg = (mimic_combo2KS[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo5KS[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo8KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3
mimic_ps3avg = (mimic_combo3KS[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo6KS[['Iteration', 'Time', 'Fitness','FEvals']] + mimic_combo9KS[['Iteration', 'Time', 'Fitness','FEvals']]) / 3

# Concatenate These

# Common Problem Size 1; Different Seeds
mimic_ps1_KS = pd.concat([mimic_combo1KS, mimic_combo4KS, mimic_combo7KS, mimic_ps1avg], axis=1)

# Common Problem Size 2; Different Seeds
mimic_ps2_KS = pd.concat([mimic_combo2KS, mimic_combo5KS, mimic_combo8KS, mimic_ps2avg], axis=1)

# Common Problem Size 3; Different Seeds
mimic_ps3_KS = pd.concat([mimic_combo3KS, mimic_combo6KS, mimic_combo9KS, mimic_ps3avg], axis=1)

mimic_list = [# FOUR PEAKS
mimic_ps1_FP,
mimic_ps2_FP,
mimic_ps3_FP,

# FLIP FLOP
mimic_ps1_FF,
mimic_ps2_FF,
mimic_ps3_FF,

# KNAPmimicCK
mimic_ps1_KS,
mimic_ps2_KS,
mimic_ps3_KS]

for mimic in mimic_list:
    mimic.columns = ['s1_Iteration', 's1_Time', 's1_Fitness', 's1_FEvals', 's1_use_fast_mimic', 's1_Population Size', 's1_Keep Percent', 's1_max_iters', 's1_fast_mimic,',
                     's2_Iteration', 's2_Time', 's2_Fitness', 's2_FEvals', 's2_use_fast_mimic', 's2_Population Size', 's2_Keep Percent', 's2_max_iters', 's2_fast_mimic,',
                     's3_Iteration', 's3_Time', 's3_Fitness', 's3_FEvals', 's3_use_fast_mimic', 's3_Population Size', 's3_Keep Percent', 's3_max_iters', 's3_fast_mimic,',
                      'avg_Iteration', 'avg_Time', 'avg_Fitness', 'avg_FEvals']

### Plot Optimization Problem

$$
\text{Plot RHC Seed Averages}
$$

In [955]:
# # FOUR PEAKS
# rhc_ps1_FP
# rhc_ps2_FP
# rhc_ps3_FP

# # FLIP FLOP
# rhc_ps1_FF
# rhc_ps2_FF
# rhc_ps3_FF

# # KNAPSACK
# rhc_ps1_KS
# rhc_ps2_KS
# rhc_ps3_KS

In [ ]:
x = 'Iteration'
y = 'Fitness'
prob = 'Knapsack'
param = 'CR'
size = '_size25'
for j in range(1, 16):
    fig, axes = plt.subplots(nrows=1,
                            ncols=1,
                            figsize=(10, 5))

    b1 = (rhc_ps1_KS['s1_current_restart'] == j)
    b2 = (rhc_ps1_KS['s2_current_restart'] == j)
    b3 = (rhc_ps1_KS['s3_current_restart'] == j)
    rhcKS = rhc_ps1_KS[b1 & b2 & b3]
    
    axes.plot(rhcKS[f'avg_{x}'], rhcKS[f'avg_{y}'])
    axes.set_xlabel(xlabel=f'{x}')
    axes.set_ylabel(ylabel=f'Avg {y}')
    #axes.text(x=250, y=15, s='scott\nscott')
    title = f'RHC; Size: 25; {prob}; CR: {j}'
    axes.set_title(label=title)
    home_loc = f"/home/leonardo_leads/Documents/SchoolDocs/"
    gt_loc = f'{home_loc}ga_tech_masters/omscs_ml/spring2024/'
    course_assign = f'{gt_loc}cs7641_machine_learning/lectures/week9/assignment2/source/'
    save_loc = f'{course_assign}plots/opt_prob/Random Hill Climbing/{prob}/{param}{j}{size}.png'
    plt.savefig(fname=f"{save_loc}")
    # plt.show()

$$ \text{Plot SA Seed Averages}
$$

In [957]:
# # FOUR PEAKS
# sa_ps1_FP
# sa_ps2_FP
# sa_ps3_FP

# # FLIP FLOP
# sa_ps1_FF
# sa_ps2_FF
# sa_ps3_FF

# # KNAPsaCK
# sa_ps1_KS
# sa_ps2_KS
# sa_ps3_KS

In [ ]:
x = 'Iteration'
y = 'Fitness'
prob = 'Four Peaks'
param = 'Temp'
size = '_size25'
for j in [1, 10, 50, 100, 200, 250]:
    fig, axes = plt.subplots(nrows=1,
                            ncols=1,
                            figsize=(10, 7))
    
    b1 = (sa_ps1_FP['s1_Temperature'] == j)
    b2 = (sa_ps1_FP['s2_Temperature'] == j)
    b3 = (sa_ps1_FP['s3_Temperature'] == j)
    saFP = sa_ps1_FP[b1 & b2 & b3]
    
    axes.plot(saFP[f'avg_{x}'], saFP[f'avg_{y}'])
    axes.set_xlabel(xlabel=f'{x}')
    axes.set_ylabel(ylabel=f'Avg {y}')
    #axes.text(x=250, y=25, s='scott\nscott')
    title = f'sa; Size: 25; {prob}; Temperature: {j}'
    axes.set_title(label=title)
    # save_loc = f"./plots/nf/{model_name}/"
    # plt.savefig(fname=f"{save_loc}{iters}.png")
    home_loc = f"/home/leonardo_leads/Documents/SchoolDocs/"
    gt_loc = f'{home_loc}ga_tech_masters/omscs_ml/spring2024/'
    course_assign = f'{gt_loc}cs7641_machine_learning/lectures/week9/assignment2/source/'
    save_loc = f'{course_assign}plots/opt_prob/Simulated Annealing/{prob}/{param}{j}{size}.png'
    plt.savefig(fname=f"{save_loc}")
    #plt.show()

$$ \text{Plot GA Seed Averages}
$$

In [959]:
# # FOUR PEAKS
# ga_ps1_FP
# ga_ps2_FP
# ga_ps3_FP

# # FLIP FLOP
# ga_ps1_FF
# ga_ps2_FF
# ga_ps3_FF

# # KNAPSACK
# ga_ps1_KS
# ga_ps2_KS
# ga_ps3_KS

In [ ]:
x = 'Iteration'
y = 'Fitness'
prob = 'Knapsack'
param1 = 'Pop Size'
param2 = 'Mut Rate'
size = '_size25'
for i in [100, 150, 200]:
    for j in [0.4, 0.5, 0.6]:
        fig, axes = plt.subplots(nrows=1,
                                ncols=1,
                                figsize=(10, 7))

        b1 = (ga_ps1_KS[f's1_Population Size'] == i) & (ga_ps1_KS[f's1_Mutation Rate'] == j)
        b2 = (ga_ps1_KS[f's2_Population Size'] == i) & (ga_ps1_KS[f's2_Mutation Rate'] == j)
        b3 = (ga_ps1_KS[f's3_Population Size'] == i) & (ga_ps1_KS[f's3_Mutation Rate'] == j)
        gaKS = ga_ps1_KS[(b1) & (b2) & (b3)]
        
        axes.plot(gaKS[f'avg_{x}'], gaKS[f'avg_{y}'])
        axes.set_xlabel(xlabel=f'{x}')
        axes.set_ylabel(ylabel=f'Avg {y}')
        #axes.text(x=250, y=25, s='scott\nscott')
        title = f'ga; Size: 25; {prob}; PS: {i}; MR: {j}'
        axes.set_title(label=title)
        home_loc = f"/home/leonardo_leads/Documents/SchoolDocs/"
        gt_loc = f'{home_loc}ga_tech_masters/omscs_ml/spring2024/'
        course_assign = f'{gt_loc}cs7641_machine_learning/lectures/week9/assignment2/source/'
        save_loc = f'{course_assign}plots/opt_prob/Genetic Algorithm/{prob}/{param1}{i}_{param2}{j}{size}.png'
        plt.savefig(fname=f"{save_loc}")
        plt.show()

$$ \text{Plot MIMIC Seed Averages}
$$

In [961]:
# # FOUR PEAKS
# mimic_ps1_FP
# mimic_ps2_FP
# mimic_ps3_FP

# # FLIP FLOP
# mimic_ps1_FF
# mimic_ps2_FF
# mimic_ps3_FF

# # KNAPSACK
# mimic_ps1_KS
# mimic_ps2_KS
# mimic_ps3_KS

In [ ]:
x = 'Iteration'
y = 'Fitness'
prob = 'Four Peaks'
param1 = 'KP'
param2 = 'PS'
size = '_size25'
for i in [0.25, 0.5, 0.75]:
    for j in [150, 200, 250]:
        fig, axes = plt.subplots(nrows=1,
                                ncols=1,
                                figsize=(10, 7))
        
        b1 = (mimic_ps1_FP[f's1_Keep Percent'] == i) & (mimic_ps1_FP[f's1_Population Size'] == j)
        b2 = (mimic_ps1_FP[f's2_Keep Percent'] == i) & (mimic_ps1_FP[f's2_Population Size'] == j)
        b3 = (mimic_ps1_FP[f's3_Keep Percent'] == i) & (mimic_ps1_FP[f's3_Population Size'] == j)
        mimicFP = mimic_ps1_FP[(b1) & (b2) & (b3)]
        
        axes.plot(mimicFP[f'avg_{x}'], mimicFP[f'avg_{y}'])
        axes.set_xlabel(xlabel=f'{x}')
        axes.set_ylabel(ylabel=f'Avg {y}')
        #axes.text(x=250, y=25, s='scott\nscott')
        title = f'mimic; Size: 25; {prob}; KP: {i}; PS: {j}'
        axes.set_title(label=title)
        home_loc = f"/home/leonardo_leads/Documents/SchoolDocs/"
        gt_loc = f'{home_loc}ga_tech_masters/omscs_ml/spring2024/'
        course_assign = f'{gt_loc}cs7641_machine_learning/lectures/week9/assignment2/source/'
        save_loc = f'{course_assign}plots/opt_prob/MIMIC/{prob}/{param1}{i}{param2}{j}{size}.png'
        plt.savefig(fname=f"{save_loc}")
        plt.show()